In [0]:
# 04_pipeline_logs_viewer

from pyspark.sql.functions import col, to_date, current_date, desc
from datetime import datetime

print(f"📊 Оновлення логів: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

# ===  логи из Delta таблиці ===
logs_df = spark.table("pipeline_logs")

# === останні 20 логів ===
print("🕒 останні 20 записів логів:")
display(logs_df.orderBy(desc("timestamp")).limit(20))


# === Агрегація: кількість шагів по статусу и шарам за останні 7 днів ===
agg_df = logs_df \
    .withColumn("log_date", to_date(col("timestamp"))) \
    .filter(col("log_date") >= to_date(current_date()) - 7) \
    .groupBy("layer", "step", "status") \
    .count() \
    .orderBy("layer", "step", "status")

print("📈 Агрегація логів за останні 7 днів:")
display(agg_df)


# === помилки (якщо були) ===
print("❌ помилки пайплайна:")
errors_df = logs_df.filter(col("status").startswith("error")).orderBy(desc("timestamp"))
display(errors_df)


📊 Оновлення логів: 2025-05-31 12:31:34
🕒 останні 20 записів логів:


step,layer,timestamp,status,rows_processed,comment
aggregation,gold,2025-05-31 12:26:34,success,4,Агрегація середнього Close та сумарного Volume по тікеру и сектору
transformation,silver,2025-05-31 12:20:02,success,1000,поєднання та очистка данних з бронзового шару
ingestion,bronze,2025-05-31 12:12:54,success,250,BAC (Financials)
ingestion,bronze,2025-05-31 12:12:51,success,250,JPM (Financials)
ingestion,bronze,2025-05-31 12:12:49,success,250,MSFT (Technology)
ingestion,bronze,2025-05-31 12:12:47,success,250,AAPL (Technology)
aggregation,gold,2025-05-31 05:03:38,success,4,Агрегация среднего Close и суммарного Volume по тикеру и сектору
transformation,silver,2025-05-31 05:02:59,success,1000,Объединение и очистка данных из бронзового слоя
ingestion,bronze,2025-05-31 05:02:19,success,250,BAC (Financials)
ingestion,bronze,2025-05-31 05:02:17,success,250,JPM (Financials)


📈 Агрегація логів за останні 7 днів:


layer step status count bronze ingestion error: [DELTA_INVALID_CHARACTERS_IN_COLUMN_NAMES] Found invalid character(s) among ' ,;{}()\n\t=' in the column names of your schema.
Invalid column names: Stock Splits.
Please use other characters and try again.
Alternatively, enable Column Mapping to keep using these characters.

JVM stacktrace:
com.databricks.sql.transaction.tahoe.DeltaAnalysisException
	at com.databricks.sql.transaction.tahoe.DeltaErrorsBase.foundInvalidCharsInColumnNames(DeltaErrors.scala:2365)
	at com.databricks.sql.transaction.tahoe.DeltaErrorsBase.foundInvalidCharsInColumnNames$(DeltaErrors.scala:2364)
	at com.databricks.sql.transaction.tahoe.DeltaErrors$.foundInvalidCharsInColumnNames(DeltaErrors.scala:3948)
	at com.databricks.sql.transaction.tahoe.schema.SchemaUtils$.checkSchemaFieldNames(SchemaUtils.scala:1366)
	at com.databricks.sql.transaction.tahoe.OptimisticTransactionImpl.assertMetadata(OptimisticTransaction.scala:1018)
	at com.databricks.sql.transaction.tahoe.OptimisticTransactionImpl.assertMetadata$(OptimisticTransaction.scala:1001)
	at com.databricks.sql.transaction.tahoe.OptimisticTransaction.assertMetadata(OptimisticTransaction.scala:196)
	at com.databricks.sql.transaction.tahoe.OptimisticTransactionImpl.updateMetadataInternal(OptimisticTransaction.scala:894)
	at com.databricks.sql.transaction.tahoe.OptimisticTransactionImpl.updateMetadataInternal$(OptimisticTransaction.scala:637)
	at com.databricks.sql.transaction.tahoe.OptimisticTransaction.updateMetadataInternal(OptimisticTransaction.scala:196)
	at com.databricks.sql.transaction.tahoe.OptimisticTransactionImpl.updateMetadata(OptimisticTransaction.scala:613)
	at com.databricks.sql.transaction.tahoe.OptimisticTransactionImpl.updateMetadata$(OptimisticTransaction.scala:606)
	at com.databricks.sql.transaction.tahoe.OptimisticTransaction.updateMetadata(OptimisticTransaction.scala:196)
	at com.databricks.sql.transaction.tahoe.schema.ImplicitMetadataOperation.updateMetadata(ImplicitMetadataOperation.scala:150)
	at com.databricks.sql.transaction.tahoe.schema.ImplicitMetadataOperation.updateMetadata$(ImplicitMetadataOperation.scala:91)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDeltaEdge.updateMetadata(WriteIntoDeltaEdge.scala:111)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDeltaEdge.writeAndReturnCommitDataAndMaterializationPlans(WriteIntoDeltaEdge.scala:313)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDeltaEdge.writeAndReturnCommitData(WriteIntoDeltaEdge.scala:209)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDeltaEdge.$anonfun$run$2(WriteIntoDeltaEdge.scala:155)
	at com.databricks.sql.transaction.tahoe.DeltaLog.withNewTransaction(DeltaLog.scala:302)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDeltaEdge.$anonfun$run$1(WriteIntoDeltaEdge.scala:142)
	at com.databricks.sql.acl.CheckPermissions$.$anonfun$trusted$2(CheckPermissions.scala:2414)
	at com.databricks.sql.util.ThreadLocalTagger.withTag(QueryTagger.scala:63)
	at com.databricks.sql.util.ThreadLocalTagger.withTag$(QueryTagger.scala:60)
	at com.databricks.sql.util.QueryTagger$.withTag(QueryTagger.scala:212)
	at com.databricks.sql.acl.CheckPermissions$.trusted(CheckPermissions.scala:2414)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDeltaEdge.run(WriteIntoDeltaEdge.scala:141)
	at com.databricks.sql.transaction.tahoe.sources.DeltaDataSource.createRelation(DeltaDataSource.scala:286)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:55)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.$anonfun$sideEffectResult$2(commands.scala:84)
	at org.apache.spark.sql.execution.SparkPlan.runCommandInAetherOrSpark(SparkPlan.scala:194)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.$anonfun$sideEffectResult$1(commands.scala:84)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala:94)
	at org.apache.spark.sql.execution.command

❌ помилки пайплайна:


step layer timestamp status rows_processed comment ingestion bronze 2025-05-26 14:50:19 error: [DELTA_INVALID_CHARACTERS_IN_COLUMN_NAMES] Found invalid character(s) among ' ,;{}()\n\t=' in the column names of your schema.
Invalid column names: Stock Splits.
Please use other characters and try again.
Alternatively, enable Column Mapping to keep using these characters.

JVM stacktrace:
com.databricks.sql.transaction.tahoe.DeltaAnalysisException
	at com.databricks.sql.transaction.tahoe.DeltaErrorsBase.foundInvalidCharsInColumnNames(DeltaErrors.scala:2365)
	at com.databricks.sql.transaction.tahoe.DeltaErrorsBase.foundInvalidCharsInColumnNames$(DeltaErrors.scala:2364)
	at com.databricks.sql.transaction.tahoe.DeltaErrors$.foundInvalidCharsInColumnNames(DeltaErrors.scala:3948)
	at com.databricks.sql.transaction.tahoe.schema.SchemaUtils$.checkSchemaFieldNames(SchemaUtils.scala:1366)
	at com.databricks.sql.transaction.tahoe.OptimisticTransactionImpl.assertMetadata(OptimisticTransaction.scala:1018)
	at com.databricks.sql.transaction.tahoe.OptimisticTransactionImpl.assertMetadata$(OptimisticTransaction.scala:1001)
	at com.databricks.sql.transaction.tahoe.OptimisticTransaction.assertMetadata(OptimisticTransaction.scala:196)
	at com.databricks.sql.transaction.tahoe.OptimisticTransactionImpl.updateMetadataInternal(OptimisticTransaction.scala:894)
	at com.databricks.sql.transaction.tahoe.OptimisticTransactionImpl.updateMetadataInternal$(OptimisticTransaction.scala:637)
	at com.databricks.sql.transaction.tahoe.OptimisticTransaction.updateMetadataInternal(OptimisticTransaction.scala:196)
	at com.databricks.sql.transaction.tahoe.OptimisticTransactionImpl.updateMetadata(OptimisticTransaction.scala:613)
	at com.databricks.sql.transaction.tahoe.OptimisticTransactionImpl.updateMetadata$(OptimisticTransaction.scala:606)
	at com.databricks.sql.transaction.tahoe.OptimisticTransaction.updateMetadata(OptimisticTransaction.scala:196)
	at com.databricks.sql.transaction.tahoe.schema.ImplicitMetadataOperation.updateMetadata(ImplicitMetadataOperation.scala:150)
	at com.databricks.sql.transaction.tahoe.schema.ImplicitMetadataOperation.updateMetadata$(ImplicitMetadataOperation.scala:91)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDeltaEdge.updateMetadata(WriteIntoDeltaEdge.scala:111)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDeltaEdge.writeAndReturnCommitDataAndMaterializationPlans(WriteIntoDeltaEdge.scala:313)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDeltaEdge.writeAndReturnCommitData(WriteIntoDeltaEdge.scala:209)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDeltaEdge.$anonfun$run$2(WriteIntoDeltaEdge.scala:155)
	at com.databricks.sql.transaction.tahoe.DeltaLog.withNewTransaction(DeltaLog.scala:302)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDeltaEdge.$anonfun$run$1(WriteIntoDeltaEdge.scala:142)
	at com.databricks.sql.acl.CheckPermissions$.$anonfun$trusted$2(CheckPermissions.scala:2414)
	at com.databricks.sql.util.ThreadLocalTagger.withTag(QueryTagger.scala:63)
	at com.databricks.sql.util.ThreadLocalTagger.withTag$(QueryTagger.scala:60)
	at com.databricks.sql.util.QueryTagger$.withTag(QueryTagger.scala:212)
	at com.databricks.sql.acl.CheckPermissions$.trusted(CheckPermissions.scala:2414)
	at com.databricks.sql.transaction.tahoe.commands.WriteIntoDeltaEdge.run(WriteIntoDeltaEdge.scala:141)
	at com.databricks.sql.transaction.tahoe.sources.DeltaDataSource.createRelation(DeltaDataSource.scala:286)
	at org.apache.spark.sql.execution.datasources.SaveIntoDataSourceCommand.run(SaveIntoDataSourceCommand.scala:55)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.$anonfun$sideEffectResult$2(commands.scala:84)
	at org.apache.spark.sql.execution.SparkPlan.runCommandInAetherOrSpark(SparkPlan.scala:194)
	at org.apache.spark.sql.execution.command.ExecutedCommandExec.$anonfun$sideEffectResult$1(commands.scala:84)
	at com.databricks.spark.util.FrameProfiler$.record(FrameProfiler.scala